**Treinamento da inteligencia artificial**

In [ ]:
# Checagem para ver se a GPU está sendo detectada
!nvidia-smi

In [ ]:
#Mostra os arquivos da pasta local
!ls

**1) Baixa o Darknet**



In [ ]:
#baixa na pasta local o Darknet
!git clone https://github.com/AlexeyAB/darknet

**2) Compila o Darknet para habilitar o treino com GPU**


In [ ]:
# altera o makefile para utilizar opencv e GPU
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configuração para treino do YOLOv3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 20000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Cria pasta para salvar os treinos anteriores de 1.000 em 1.000, até atingir 20.0000
!mkdir "/home/matheus/Downloads/weights"

In [ ]:
!echo "Car" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /home/matheus/Downloads/weights' > data/obj.data
!mkdir data/obj

In [ ]:
# Baixa o modelo pré treinado: weights darknet model 53 para realizar o novo treino
!wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extrai as imagens**

As imagens estão zipadas na pasta Downloads com o nome imagens.zip

In [ ]:
!unzip /home/matheus/Downloads/imagens.zip -d data/obj

In [ ]:
# Realiza os ultimos ajustes no label das imagens antes de iniciar o treinamento
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Inicia o treino**

In [ ]:
#comando para iniciar o treino utilizando as configurações feitas a cima
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show